In [3]:
from transform import prepare_columns_to_excel, save_dataframe_to_excel_tab

In [11]:
from extract import get_interim_data

df = get_interim_data()
df['party'].unique()

array([None, 'no party', 'MDB', 'PP', 'PSDB', 'PSD', 'PPS', 'DEM', 'PMDB',
       'PV', 'PSB', 'PODE', 'no party ', 'Republicanos', 'NOVO', 'PSL',
       'PL', 'PT', 'União Brasil', 'PCdoB', 'PRD', 'PDT', 'REDE', 'PSOL',
       'PDB'], dtype=object)

In [4]:
# Selecting only cabinet
ministerios_condition = df['category_(bureaucracy_or_cabinet)'] == "Ministério"
df_ministerios = df[ministerios_condition]# Second version of table

# Columns for the desired table 
desired_cols = [
    'year',
    'agency',
    "perc_concedido_e_parcialmente",
]
df_ministerios_perc_conc_parc = df_ministerios[desired_cols]

# Pivot table to aggregate multiple values
pivoted_df = df_ministerios_perc_conc_parc.pivot_table(index='agency', columns='year', values='perc_concedido_e_parcialmente', aggfunc='mean')

# Changing column names to str
pivoted_df.columns = pivoted_df.columns.astype(str)

# Flattening df to remove multi-level columns
flatenned_df = pivoted_df.reset_index()

# Changing column names to match excel
flatenned_df_to_excel = prepare_columns_to_excel(flatenned_df)

# Checking
flatenned_df_to_excel.head(5)

year,Agency,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,AGU – Advocacia-Geral da União,0.511864,0.471535,0.515152,0.454340,0.582129,0.508238,0.714286,0.593391,0.805255,0.818182
1,CC-PR – Casa Civil da Presidência da República,0.841880,0.770484,0.641379,0.634328,0.809976,0.685714,0.555556,0.577381,0.647639,0.701126
2,CGU – Controladoria-Geral da União,0.684647,0.741766,0.741969,0.717949,0.719971,0.713474,0.699789,0.672182,0.736473,0.726799
3,GSI-PR – Gabinete de Segurança Institucional d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.326772,0.384615
4,MAPA - Ministério da Agricultura e Pecuária,0.860215,0.839797,0.883135,0.872321,0.857547,0.689548,0.695215,0.683855,0.784294,0.766883


Dataset with the party for each agency in each year

In [5]:
# Selecting desired columns
desired_cols = [
    'year',
    'agency',
    "party"
]
df_ministerios_perc_conc_parc = df_ministerios[desired_cols]

# Removing agencies with no parties
df_ministerios_perc_conc_parc = df_ministerios_perc_conc_parc[df_ministerios_perc_conc_parc['party'] != 'no party']
df_ministerios_perc_conc_parc = df_ministerios_perc_conc_parc[df_ministerios_perc_conc_parc['party'].astype(str) != 'None']

# Renaimng MDB to PMDB in the party column
df_ministerios_perc_conc_parc.loc[df_ministerios_perc_conc_parc['party'] == 'MDB', 'party'] = 'PMDB'

# Preparing to excel
df_ministerios_perc_conc_parc_to_excel = prepare_columns_to_excel(df_ministerios_perc_conc_parc)

# Checking
df_ministerios_perc_conc_parc_to_excel.head(5)

,Year,Agency,Party
1074,2017,CC-PR – Casa Civil da Presidência da República,PMDB
1077,2017,MAPA - Ministério da Agricultura e Pecuária,PP
1078,2017,MCID - Ministério das Cidades,PSDB
1079,2017,MCID - Ministério das Cidades,PP
1080,2017,"MCTI – Ministério da Ciência, Tecnologia, Inov...",PSD


In [13]:
from utils import write_to_excel
from extract import RAW_DIR

copy_df = df.copy()
copy_df_excel = prepare_columns_to_excel(copy_df)
write_to_excel(copy_df, RAW_DIR / 'bana(editada).xlsx', 'Cabinet & Bureaucracy Correct')